# The Role of Summary Statistics in SBI

**Time: ~20 minutes**

In Notebook 3, we learned how to use the `sbi` toolbox for Neural Posterior Estimation. Now we'll explore a crucial question:

> **How do summary statistics affect the quality of our inference?**

In simulation-based inference, we often need to compress high-dimensional simulator outputs (like time series) into lower-dimensional summary statistics. The choice of summary statistics can dramatically impact inference quality!

## What We'll Explore

1. **Simple statistics**: Mean, std, max, skew, kurtosis (10 summary stats)
2. **Advanced statistics**: Adding autocorrelation to capture temporal dynamics (20 summary stats)
3. **Neural embedding**: Let a neural network learn the summary statistics directly from raw time series

**Key insight**: More informative summary statistics → tighter posteriors → better parameter recovery!

**Documentation**: [sbi.readthedocs.io](https://sbi.readthedocs.io/)

In [ ]:
import matplotlib.pyplot as plt
import torch
import numpy as np

from sbi.inference import NPE
from sbi.analysis import pairplot
from sbi.neural_nets import posterior_nn
from sbi.neural_nets.embedding_nets import CausalCNNEmbedding

from simulators import (
    create_lotka_volterra_prior,
    generate_observed_data,
    lotka_volterra_simulator,
    simulate,
)

# For reproducibility
torch.manual_seed(42)
np.random.seed(42)

---
## Setup: The Lotka-Volterra Predator-Prey Model

We continue working with the Lotka-Volterra model from the previous notebooks. This model describes the dynamics of a predator-prey ecosystem with 4 parameters.

In [ ]:
# Setup the prior (same for all experiments)
prior = create_lotka_volterra_prior()

# True parameters for our "observed" data
_, theta_o = generate_observed_data()  # theta_o = [0.1, 0.02, 0.01, 0.1]


In [ ]:
# Visualize the "ground truth" time series
time = np.arange(0, 200, 0.1)
ts_observed = simulate(theta_o.numpy())

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

ax = axes[0]
ax.plot(time, ts_observed[:, 0], color="C0", linewidth=2)
ax.set_xlabel("Time (days)")
ax.set_ylabel("Population")
ax.set_title("Prey Dynamics (Ground Truth)")

ax = axes[1]
ax.plot(time, ts_observed[:, 1], color="C1", linewidth=2)
ax.set_xlabel("Time (days)")
ax.set_ylabel("Population")
ax.set_title("Predator Dynamics (Ground Truth)")

plt.tight_layout()
plt.show()

print(f"Time series length: {len(ts_observed)} time points")
print("!! The challenge: How do we compress this into informative summary statistics? !!")

In [ ]:
# Helper function for posterior predictive visualization
def plot_posterior_predictive(samples, theta_o, ts_observed, time, n_samples=50, title=""):
    """Plot posterior predictive simulations vs ground truth."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 4))

    # Sample some posterior parameters and simulate
    indices = np.random.choice(len(samples), size=n_samples, replace=False)

    for idx in indices:
        theta_sample = samples[idx].numpy()
        ts_sample = simulate(theta_sample)
        axes[0].plot(time, ts_sample[:, 0], color="C0", alpha=0.15, linewidth=0.5)
        axes[1].plot(time, ts_sample[:, 1], color="C1", alpha=0.15, linewidth=0.5)

    # Plot ground truth
    axes[0].plot(time, ts_observed[:, 0], color="black", linewidth=2, label="Ground truth")
    axes[1].plot(time, ts_observed[:, 1], color="black", linewidth=2, label="Ground truth")

    # Formatting
    axes[0].set_xlabel("Time (days)")
    axes[0].set_ylabel("Population")
    axes[0].set_title(f"Prey - {title}")
    axes[0].legend()
    axes[0].plot([], [], color="C0", alpha=0.5, label="Posterior samples")
    axes[0].legend()

    axes[1].set_xlabel("Time (days)")
    axes[1].set_ylabel("Population")
    axes[1].set_title(f"Predator - {title}")
    axes[1].plot([], [], color="C1", alpha=0.5, label="Posterior samples")
    axes[1].legend()

    plt.tight_layout()
    plt.show()

---
## Think First!

Before we start experimenting, let's think about summary statistics:

**Question 1**: Why do we need summary statistics in SBI? Why not use the raw time series directly?

**Question 2**: What properties should "good" summary statistics have?

**Question 3**: What information might simple statistics (mean, std, max) miss about oscillatory dynamics?

<details>
<summary>Click to reveal answers</summary>

1. **Why summary statistics?**
   - Raw simulator output can be very high-dimensional (e.g., 2000 time points × 2 populations = 4000 dimensions!)
   - Neural networks struggle with such high-dimensional inputs
   - Well-chosen summary statistics can capture the essential information in a compact form
   - However: poor summary statistics can lose crucial information!

2. **Good summary statistics should:**
   - Be **sufficient**: capture all information relevant for distinguishing parameters
   - Be **low-dimensional**: easier for the neural network to learn
   - Be **robust**: not too sensitive to noise
   - Be **informative**: different parameters should produce different statistics

3. **What simple statistics miss:**
   - **Temporal structure**: mean/std don't capture oscillation frequency or phase
   - **Autocorrelation**: how values at time t relate to values at time t+k
   - **Periodicity**: the characteristic period of the predator-prey cycles
   - This is why we'll add autocorrelation features as "advanced" statistics!

</details>

---
## Part 1: Simple Summary Statistics

Let's start with basic summary statistics: **mean, std, max, skew, and kurtosis** for each population.

This gives us **10 summary statistics** total (5 per population).

In [ ]:
# Generate observed data with SIMPLE summary statistics (no autocorrelation)
x_o_simple, _ = generate_observed_data(use_autocorrelation=False)

print(f"with x_o_simple we have {x_o_simple.shape[1]} summary statistics")

### Your Task: Train NPE with Simple Statistics

Generate training data and train NPE using the simple summary statistics.

**Hints**:
- Use `lotka_volterra_simulator(theta, use_autocorrelation=False)` for simple stats
- Use the same workflow as in Notebook 3: sample -> simulate -> train -> sample posterior

In [ ]:
# SGenerate training data with simple statistics
num_simulations = 2_000

# TODO: Sample parameters from the prior
# theta_simple = ...

# TODO: Simulate with simple summary statistics
# x_simple = ...

print(f"Training data shapes: theta={theta_simple.shape}, x={x_simple.shape}")

In [ ]:
# Train NPE with simple statistics
# TODO: Create NPE, append simulations, and train
# npe_simple = ...

In [ ]:
# TODO: Build posterior and sample
# posterior_simple = ...
# samples_simple = ...

print(f"Posterior samples shape: {samples_simple.shape}")

### Evaluate Part 1: How good are our results?

Let's visualize the posterior and check if simulations from posterior samples match the observed data.

In [ ]:
# Pairplot for Part 1: Simple Statistics
param_labels = [r"$\alpha$", r"$\beta$", r"$\delta$", r"$\gamma$"]
limits = [[0.05, 0.15], [0.01, 0.03], [0.005, 0.03], [0.005, 0.15]]

fig, axes = pairplot(
    samples_simple,
    limits=limits,
    labels=param_labels,
    figsize=(8, 8),
    points=theta_o,
)
plt.suptitle("Part 1: Posterior with Simple Statistics (10 features)", y=1.02)
plt.show()

In [ ]:
# Posterior Predictive Check for Part 1
# Do simulations from the posterior match the observed time series?
plot_posterior_predictive(
    samples_simple, theta_o, ts_observed, time,
    n_samples=50, title="Simple Statistics"
)
print("Notice: The posterior is quite broad - many different dynamics are plausible!")
print("This is because simple statistics don't capture the oscillation patterns.")

---
## Part 2: Advanced Summary Statistics (with Autocorrelation)

The Lotka-Volterra model produces **oscillatory dynamics**. Simple statistics like mean and std don't capture the temporal structure!

Let's add **autocorrelation** features that measure how the time series correlates with itself at different time lags. This captures:
- The period of oscillations
- How quickly correlations decay
- The temporal dynamics that distinguish different parameter settings

This gives us **20 summary statistics** total (10 per population: 5 moments + 5 autocorrelation lags).

In [ ]:
# Generate observed data with ADVANCED summary statistics (with autocorrelation)
x_o_advanced, _ = generate_observed_data(use_autocorrelation=True)

print(f"with x_o_advanced we have {x_o_advanced.shape[1]} summary statistics")

### Your Task: Train NPE with Advanced Statistics

Now train NPE using the advanced summary statistics with autocorrelation.

**Hint**: Change `use_autocorrelation=True` in the simulator call.

In [ ]:
# Generate training data with advanced statistics
# TODO: Sample and simulate with autocorrelation
# theta_advanced = ...
# x_advanced = ...

print(f"Training data shapes: theta={theta_advanced.shape}, x={x_advanced.shape}")

In [ ]:
# Train NPE with advanced statistics
# TODO: Create NPE, train
# npe_advanced = ...

In [ ]:
# TODO: Build posterior and sample
# posterior_advanced = ...
# samples_advanced = ...

print(f"Posterior samples shape: {samples_advanced.shape}")

### Evaluate Part 2: Did autocorrelation help?

Let's see if adding temporal information (autocorrelation) improves our inference.

In [ ]:
# Pairplot for Part 2: Advanced Statistics
fig, axes = pairplot(
    samples_advanced,
    limits=limits,
    labels=param_labels,
    figsize=(8, 8),
    points=theta_o,
)
plt.suptitle("Part 2: Posterior with Advanced Statistics (20 features)", y=1.02)
plt.show()

In [ ]:
# Posterior Predictive Check for Part 2
plot_posterior_predictive(
    samples_advanced, theta_o, ts_observed, time,
    n_samples=50, title="Advanced Statistics (with Autocorrelation)"
)
print("Much better! The posterior samples now capture the oscillation patterns.")
print("Autocorrelation features helped the network understand temporal dynamics.")

---
## Part 3: Neural Embedding Networks (Learning Summary Statistics)

What if we could **learn** the summary statistics directly from data?

Instead of hand-crafting summary statistics, we can use a **neural embedding network** that:
1. Takes the raw time series as input
2. Learns to compress it into informative features
3. Feeds those features to the posterior estimator

The posterior approximation becomes: $q_\phi\big(\theta \mid f_\lambda(x_o)\big)$ where:
- $\phi$ are the parameters of the density estimator (normalizing flow)
- $\lambda$ are the parameters of the embedding network
- Both are **jointly learned** during training!

Workflow:
1. Create the embedding network
2. Create the density estimator WITH the embedding
3. Pass to NPE

`sbi` provides several pre-configured embedding networks:
- `FCEmbedding`: Fully-connected MLP
- `CNNEmbedding`: Convolutional neural network (for images)
- `CausalCNNEmbedding`: Causal 1D CNN based on WaveNet (for time series) ← **We'll use this!**
- `PermutationInvariantEmbedding`: For iid trial-based data

See: [sbi embedding networks tutorial](https://sbi.readthedocs.io/en/stable/advanced_tutorials/04_embedding_networks.html) and [CausalCNNEmbedding source](https://github.com/sbi-dev/sbi/blob/main/sbi/neural_nets/embedding_nets/causal_cnn.py)

In [ ]:
# First, we need a simulator that returns RAW time series (not summary statistics)
# CausalCNNEmbedding expects shape: (batch_size, in_channels, num_timepoints)
def raw_timeseries_simulator(params: torch.Tensor) -> torch.Tensor:
    """Simulate and return raw time series for CausalCNN embedding."""
    params = params.unsqueeze(0) if params.ndim == 1 else params
    # simulate() returns (timepoints, 2) for each parameter set
    # Stack to get (batch, timepoints, channels), then transpose to (batch, channels, timepoints)
    ts = torch.tensor(np.stack([simulate(p.numpy()) for p in params]), dtype=torch.float32)
    return ts.permute(0, 2, 1)  # (batch, channels=2, timepoints=2000)

# Test it
test_ts = raw_timeseries_simulator(theta_o)
print(f"Raw time series shape: {test_ts.shape}")
print("  → (batch_size, in_channels=2, num_timepoints=2000)")

### Your Task: Train NPE with Causal CNN Embedding

Now train NPE using the raw time series with a **CausalCNNEmbedding** network.

The CausalCNN is based on the WaveNet architecture and uses dilated causal convolutions to capture temporal patterns at multiple scales.

**Steps**:
1. Generate training data as raw time series (shape: `batch_size × in_channels × num_timepoints`)
2. Create a `CausalCNNEmbedding` network that compresses time series to fixed-size vectors
3. Create the density estimator 
4. Create NPE 
5. Train and sample as usual

**Key parameters for `CausalCNNEmbedding`**:
- `input_shape`: Tuple with number of timepoints, e.g. `(2000,)`
- `in_channels`: Number of input channels (2 for prey + predator)
- `output_dim`: Size of the learned summary representation
- `num_conv_layers`, `kernel_size`: Control network depth and receptive field

In [ ]:
# Generate training data as raw time series
# Note: We use fewer simulations here as training with embeddings is slower
num_simulations_cnn = 2_000

# TODO: Sample parameters
# theta_cnn = ...

# TODO: Simulate raw time series
# x_cnn = ...

# Also generate observed data as raw time series
x_o_cnn = raw_timeseries_simulator(theta_o)

print(f"Training data shapes: theta={theta_cnn.shape}, x={x_cnn.shape}")
print(f"Observed data shape: {x_o_cnn.shape}")

For reference on the CausalCNN embedding network, see the [documentation](https://sbi.readthedocs.io/en/latest/reference/_autosummary/sbi.neural_nets.embedding_nets.CausalCNNEmbedding.html).

In [ ]:
# Step 1 - Create the CausalCNN embedding network
# The embedding network compresses time series → fixed-size vector
#
# Input shape:  (batch_size, in_channels=2, num_timepoints=2000)
# Output shape: (batch_size, output_dim=20)

# TODO: Create CausalCNN embedding network
# embedding_net = ...

print(f"CausalCNN Embedding network created!")
print(f"  - Input: time series of shape (batch, 2, 2000)")
print(f"  - Output: embedding of shape (batch, 20)")
print(f"  - Total parameters: {sum(p.numel() for p in embedding_net.parameters()):,}")

For reference, on how to use embedding networks, see [this](https://sbi.readthedocs.io/en/stable/advanced_tutorials/04_embedding_networks.html)

In [ ]:
# Step 2 - Create density estimator WITH the embedding network
# The embedding_net is passed to posterior_nn() and will be jointly trained!

# TODO: Create the neural density estimator with embedding
# density_estimator = posterior_nn(...)

# Step 3 - Create NPE with the custom density estimator
# TODO: Create NPE inference object. Check how to embedd the density estimator into the NPE.
# npe_cnn = ...

# Step 4 - Train (embedding and flow are learned jointly!)
print("Training NPE with CausalCNN embedding...")
print("(The embedding network learns to extract informative features from raw time series)")
# npe_cnn....

In [ ]:
# Step 5 - Build posterior and sample
# Note: We pass the RAW time series x_o_cnn - the embedding network handles compression!
# posterior_cnn = ...
# samples_cnn = ...

print(f"Posterior samples shape: {samples_cnn.shape}")
print(f"\nThe CausalCNN embedding automatically compressed the {x_o_cnn.shape[-1]}-step time series")
print(f"into a 20-dimensional learned summary!")

### Evaluate Part 3: How well did the neural embedding learn?

Let's see if the CausalCNN embedding network learned useful features from the raw time series.

In [ ]:
# Pairplot for Part 3: CausalCNN Embedding
fig, axes = pairplot(
    samples_cnn,
    limits=limits,
    labels=param_labels,
    figsize=(8, 8),
    points=theta_o,
)
plt.suptitle("Part 3: Posterior with CausalCNN Embedding (learned features)", y=1.02)
plt.show()

In [ ]:
# Posterior Predictive Check for Part 3
plot_posterior_predictive(
    samples_cnn, theta_o, ts_observed, time,
    n_samples=50, title="CausalCNN Embedding (learned)"
)
print("The CausalCNN network learned to extract relevant features from raw time series!")
print("No hand-crafted summary statistics needed.")

---
## Comparing All Three Approaches

Now let's visualize and compare the posteriors from all three methods!

In [ ]:
# Detailed pairplot comparison
fig, axes = pairplot(
    [samples_simple, samples_advanced, samples_cnn],
    limits=limits,
    labels=param_labels,
    figsize=(10, 10),
    points=theta_o,
    diag="hist",
    upper="scatter",
)
# Add legend manually
fig.legend(
    ["Simple Stats", "Advanced Stats", "CausalCNN Embedding"],
    loc="upper right",
    bbox_to_anchor=(0.95, 0.95),
)
plt.suptitle("Posterior Comparison: Effect of Summary Statistics", y=1.02, fontsize=14)
plt.show()

---
## Summary

### Key Takeaways

| Approach | Summary Stats | Pros | Cons |
|----------|---------------|------|------|
| **Simple** | Mean, std, max, skew, kurtosis | Fast, interpretable | Misses temporal structure |
| **Advanced** | + Autocorrelation lags | Captures oscillations | Requires domain knowledge |
| **Neural (CausalCNN)** | Learned from data | No hand-crafting needed | More training data required |

### The `sbi` Pattern for Embedding Networks

```python
# 1. Create embedding network
embedding_net = CausalCNNEmbedding(input_shape=(...,), in_channels=..., output_dim=...)

# 2. Create density estimator with embedding
density_estimator = posterior_nn(model="maf", embedding_net=embedding_net)

# 3. Create inference object
inferer = NPE(prior=prior, density_estimator=density_estimator)

# 4. Train (embedding and flow learned jointly!)
inferer.append_simulations(theta, x).train()
```

### Main Insights

1. **Summary statistics matter!** The choice of summary statistics dramatically affects inference quality.

2. **Domain knowledge helps**: Adding autocorrelation (which captures temporal dynamics) significantly improves inference for oscillatory systems.

3. **Neural embeddings are powerful**: When you don't know the "right" summary statistics, let a neural network learn them!

4. **Trade-offs exist**:
   - Simple stats: Fast and interpretable, but may lose information
   - Neural embeddings: Flexible, but need more data and computation

### When to Use What

- **Simple statistics**: Quick prototyping, well-understood systems
- **Domain-specific statistics**: When you know what features matter
- **Neural embeddings**: Complex data (images, time series), unknown optimal summaries

For more details, see the [sbi tutorial on embedding networks](https://sbi.readthedocs.io/en/stable/advanced_tutorials/04_embedding_networks.html) and [CausalCNNEmbedding source](https://github.com/sbi-dev/sbi/blob/main/sbi/neural_nets/embedding_nets/causal_cnn.py).

---
## Learning Goals

After completing this notebook, you should be able to:

- ✅ Explain why summary statistics are important in SBI
- ✅ Compare the effect of different summary statistics on inference quality
- ✅ Use autocorrelation features for time series data
- ✅ Implement neural embedding networks (CausalCNN) for learning summary statistics
- ✅ Make informed decisions about which summary statistics to use for your problem